[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/persimmon-persimmon/mnist-multilabel/blob/master/data_preparation.ipynb)

## マウントとインポート

In [ ]:
from google.colab import drive
import glob
import os
drive.mount("/content/drive/")
try:
    os.chdir("drive/MyDrive/mnist_multilabel/")
except:
    pass

In [ ]:
from keras.datasets import mnist
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import random
from PIL import Image

In [ ]:
(_x_train_val, _y_train_val), (_x_test, _y_test)=mnist.load_data()
_x_train, _x_val, _y_train, _y_val = train_test_split(_x_train_val, _y_train_val, test_size=0.2)
train={i:[x for x, t in zip(_x_train, _y_train) if t == i] for i in range(10)}
val={i:[x for x, t in zip(_x_val, _y_val) if t == i] for i in range(10)}
test={i:[x for x, t in zip(_x_test, _y_test) if t == i] for i in range(10)}
image_set={"train":train,"val":val,"test":test}

In [ ]:
n_image={"train":48000,"val":12000,"test":10000}
n_image={"train":7000,"val":2000,"test":1000}
n_image={"train":14000,"val":4000,"test":2000}
random.seed(0)
height = 224
width = 224
weight=[30,30,25,25,20,20,15,15,10,10]
ary=[]
for data_type in ["train","val","test"]:
    for t in range(1,n_image[data_type]+1):
        blank = np.zeros((height, width, 3))
        label=[0]*10
        for i in range(10):
            if random.random()<weight[i]/100:
                im=random.choice(image_set[data_type][i])
                x_ = int(random.uniform(0,height - 28))
                y_ = int(random.uniform(0,width - 28))
                blank[x_:x_ + 28,y_:y_ + 28,0]+=im
                blank[x_:x_ + 28,y_:y_ + 28,1]+=im
                blank[x_:x_ + 28,y_:y_ + 28,2]+=im
                label[i]=1
        filepath=f'images/{data_type}_{str(t).zfill(6)}.jpg'
        ary.append([filepath,data_type]+label)
        cv2.imwrite(filepath,blank)
df=pd.DataFrame(ary)
df.columns=["filepath","data_type"]+[str(i) for i in range(10)]
df.to_csv("data.csv",index=False)

## 画像確認

In [ ]:
idx=random.randint(0,100)
filepath=df["filepath"][idx]
with Image.open(filepath) as f:
    img = f.convert("RGB")
print("idx =",idx)
print(filepath)
print(np.arange(10))
print(df.loc[idx,[str(i) for i in range(10)]].values)
img